In [44]:
import os
import numpy as np
from tqdm import tqdm 
from PIL import Image

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
    
    images = []
    for file in tqdm(files):
        try:
            image = Image.open(path + "/" + file)
            image = image.resize((224,224), Image.LANCZOS)
            image = image.convert("RGB")

            image = np.asarray(image)

            images.append(image)
        except OSError:
            pass
        
    return images
    
    
    
    
mona_lisa = read_images("Desktop/Bilder/mona_lisa")
mona_lisa = np.asarray(mona_lisa)

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:01<00:00, 36.77it/s]


In [45]:
mona_lisa.shape

(51, 224, 224, 3)

In [46]:
import os
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop

In [47]:

X = preprocess_input(mona_lisa)

In [48]:
vgg16_model = VGG16(include_top = False,input_shape=(224,224,3))
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [49]:
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(units=4096, activation="relu"))
model.add(Dense(units=1024, activation="relu"))
model.add(Dense(units=5, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.00001),loss = "categorical_crossentropy", metrics = ["acc"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 4096)              102764544 
                                                                 
 dense_3 (Dense)             (None, 1024)              4195328   
                                                                 
 dense_4 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 121,679,685
Trainable params: 121,679,685
Non-trainable params: 0
_________________________________________________________________


In [50]:
labels = ["Michelangelo-ErschaffungAdams", "Michelangelo-Die heilige Familie", "VanGogh-Selbstbildnis"]